In [4]:
using Plots, Random, DataFrames, LinearAlgebra, Printf, JLD, Distributions, Combinatorics, MLJ, Graphs, Gurobi, JuMP, Statistics, Clustering, Distances
const GRB_ENV = Gurobi.Env()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-04


Gurobi.Env(Ptr{Nothing} @0x00000002a2e42000, false, 0)

### Generating points

In [5]:
function generate_points(K::Int64, N::Int64, D::Int64 , std::Float64, seed = 1234)
    """
    Generates points for the k-means problem

        input: 
            - I: number of clusters 
            - N: number of points
            - D: dimnesionality of points

        output:
            - points: NxD matrix of standardized points
    """
    Random.seed!(seed);
    X, yy = MLJ.make_blobs(N, D; centers=K, cluster_std=std)
    points = Matrix(DataFrame(X));
    min = minimum(points, dims=1);
    max = maximum(points, dims=1);
    points = (points .- min) ./ (max .- min);
    return points
end

generate_points (generic function with 2 methods)

In [6]:
points = generate_points(2, 20, 2, 0.01);

In [7]:
struct Cluster 
    assignments::Vector{Int64}
    centroid::Vector{Float64}
    cost::Float64
end

In [8]:
function compute_cluster_centroid_cost(assignments::Vector{Int64})
    """
    Computes the centroid and cost of a cluster given the indeces of the points in the cluster

        input: 
            - assignments: Nx1 matrix of cluster assignments

        output:
            - cost of the clustering assignment
            - Vector representing the centroid of the cluster
    """

    cluster_points = points[assignments,:]

    centroid = mean(cluster_points, dims=1)

    return sum( euclidean(cluster_points[i,:],centroid) for i in 1:size(cluster_points,1)), vec(centroid)
end

compute_cluster_centroid_cost (generic function with 1 method)

In [28]:
assignments = [4, 5, 6]

cost, centroid = compute_cluster_centroid_cost(assignments)

c = Cluster(assignments, centroid, cost)

Cluster([4, 5, 6], [0.666812617629264, 0.6652557705622788], 1.8808518733268333)

In [34]:
function compute_reduced_cost(data::Cluster, p::Vector{Float64}, q::Float64)
"""
Computes the reduced cost given a dual solution of the master problem

    input: 
        - Cluster: cluster to compute the reduced cost for
        - p: dual solution of the master problem
        - q: dual solution of the master problem

    output: reduced cost of the cluster
"""

    return data.cost - sum(p[i] for i in data.assignments) - q
end

compute_reduced_cost (generic function with 2 methods)

### Subproblem

In [35]:
function new_cluster(data::Matrix{Float64}, p::Vector{Float64}, q::Float64, K::Int64)

    N = size(data,1)

    #### Modified cost is in the objective 

    ###Subproblem
    sp_model = JuMP.Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attributes( sp_model, "OutputFlag" => 0)

    @variable(sp_model, θ)
    @variable(sp_model, a[1:N], Bin)
    @variable(sp_model, f[1:N, 1:N])#, Bin)
    @variable(sp_model, b[1:N-K+1], Bin)
    @variable(sp_model, γ[1:N,1:N,1:N-K+1])#, Bin)

    @constraint(sp_model, [i=1:N, j=1:N], f[i,j] <= a[i])
    @constraint(sp_model, [i=1:N, j=1:N], f[i,j] <= a[j])
    @constraint(sp_model, [i=1:N, j=1:N], f[i,j] >= a[i] + a[j] - 1)

    @constraint(sp_model, [i=1:N, j=1:N, l=1:N-K+1], γ[i,j,l] <= f[i,j])
    @constraint(sp_model, [i=1:N, j=1:N, l=1:N-K+1], γ[i,j,l] <= b[l])
    @constraint(sp_model, [i=1:N, j=1:N, l=1:N-K+1], γ[i,j,l] >= f[i,j] + b[l] - 1)

    @constraint(sp_model, [i=1:N], [θ; sum( 1/l .* sum( γ[i,j,l] .* (data[i,:] .- data[j,:]) for j=1:N) for l=1:N-K+1)] in SecondOrderCone()) #

    @objective(sp_model, Min, sum( θ - a[i]*p[i] for i in 1:N))

    optimize!(sp_model)

    #### Retrieve cluster from solution

    assignments = vec(findall(x->x>0.9, value.(a)))
    cost, centroid = compute_cluster_centroid_cost(assignments)
    Clust = Cluster(assignments, centroid, cost)
    reduced_cost = compute_reduced_cost(Clust, p, q)

    return Clust, reduced_cost 

end 

new_cluster (generic function with 1 method)

### Master problem